In [1]:
import os, sys
sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from src.utils.utils import *

# Dicionário de metadados

## Metadados cabeçalho

Cabeçalho Dicom - DDSM

In [ ]:
metadata_files = ['calc_case_description_test_set',
             'mass_case_description_train_set',
             'calc_case_description_train_set',
             'mass_case_description_test_set']

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_files: # Itera sobre os json de metados
    metadata_ddsm = load_json(current_meta,  "../../metadata/extracted_metadata/csv_and_dicom_DSSM")
    
    for current_meta_study in metadata_ddsm: # itera sobre os estudos
        tmp_meta = current_meta_study['metadata_dicom_files']
        
        all_tags = set()
        empty_tags = set()
        
        meta_dicom_files = tmp_meta['original'] + tmp_meta['cropped']
        
        for meta_dicom_file in meta_dicom_files: # Itera sobre os arquivos dicom no estudo
            for key, value in meta_dicom_file.items(): # Iterar sobre as tags no arquivo dicom
                all_tags = all_tags | set(meta_dicom_file.keys()) # união
                empty_tags = empty_tags | set([key for key, value in meta_dicom_file.items() 
                                               if meta_dicom_file[key] == ""]) # união
        update_count_tag(all_tags, dictionary_metadata)
        update_count_tag(empty_tags, dictionary_empty_meta) 
        
for key in dictionary_metadata.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata[key] -= dictionary_empty_meta[key]
        
save_json("dictionary_header_DDSM", dictionary_metadata, f"../../metadata/dictionary_dicom")

Cabeçalho Dicom - CMMD

In [ ]:
metadata_cmmd = load_json("csv_and_dicom_CMMD", "../../metadata/extracted_metadata")

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_cmmd: # Iterar sobre os estudos
    meta_dicom_files = current_meta['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    for current_meta_dicom in meta_dicom_files: # Iterar sobre os arquivos dicom no estudo
        for key, value in current_meta_dicom.items(): # Iterar sobre as tags no arquivo dicom
            all_tags = all_tags | set(current_meta_dicom.keys()) # união
            empty_tags = empty_tags | set([key for key, value in current_meta_dicom.items() 
                                           if current_meta_dicom[key] == ""]) # união
    
            if isinstance(value, list):
                for seq_item in value:
                    all_tags = all_tags | set(seq_item.keys()) # união
                    empty_tags = empty_tags | set([key for key, value in seq_item.items() 
                                           if seq_item[key] == ""]) # união
                    
    update_count_tag(all_tags, dictionary_metadata)
    update_count_tag(empty_tags, dictionary_empty_meta) 
    
for key in dictionary_metadata.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata[key] -= dictionary_empty_meta[key]

save_json("dictionary_header_CMMD", dictionary_metadata, f"../../metadata/dictionary_dicom") 

Cabelçalho Dicom - INBREAST

In [ ]:
metadata_inbreast = load_json("csv_and_dicom_INBREAST", "../../metadata/extracted_metadata")

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_inbreast: # Iterar sobre os estudos
    meta_dicom_file = current_meta['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    for key, value in meta_dicom_file.items(): # Iterar sobre as tags no arquivo dicom
        all_tags = all_tags | set(meta_dicom_file.keys()) # união
        empty_tags = empty_tags | set([key for key, value in meta_dicom_file.items() 
                                       if meta_dicom_file[key] == ""]) # união
            
    update_count_tag(all_tags, dictionary_metadata)
    update_count_tag(empty_tags, dictionary_empty_meta) 
    
for key in dictionary_metadata.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata[key] -= dictionary_empty_meta[key]

save_json("dictionary_header_INBREAST", dictionary_metadata, f"../../metadata/dictionary_dicom")

Cabeçalho PGM - MIAS

In [ ]:
mias_metadata = load_json("txt_and_pgm_MIAS", "../../metadata/extracted_metadata")
dictionary_metadata = {}

for current_meta in mias_metadata:
    metadata_pgm_file = current_meta['metadata_pgm_file']
    """ normalized_dict = {}
    
    #Criar dicionário com tags normalizada
    normalized_dict["rows (0028, 0010)"] = metadata_pgm_file['rows']
    normalized_dict["columns (0028, 0011)"] = metadata_pgm_file['columns']
    
    bits_used = get_bits_allocated(metadata_pgm_file['max_value'])
    normalized_dict["bits_allocated (0028, 0100)"] = bits_used
    normalized_dict["bits_stored (0028, 0101)"] = bits_used
    normalized_dict["high_bit (0028, 0102)"] = bits_used - 1"""
    
    for key, value in metadata_pgm_file.items():
        if value == "" or key == "img_path":
            del metadata_pgm_file[key]
    
    update_count_tag(metadata_pgm_file.keys(), dictionary_metadata)
    
save_json("dictionary_header_MIAS", dictionary_metadata, f"../../metadata/dictionary_dicom")

Juntar Dicionários de tags do cabelhaço Dicom - CMMD, DDSM e INBREAST

In [ ]:
dictionary_cmmd = load_json("dictionary_header_CMMD", "../../metadata/dictionary_dicom")
dictionary_ddsm = load_json("dictionary_header_DDSM", "../../metadata/dictionary_dicom")
dictionary_inbreast = load_json("dictionary_header_INBREAST", "../../metadata/dictionary_dicom")

full_dictionary = merge_dictionary(dictionary_cmmd, dictionary_ddsm)
full_dictionary = merge_dictionary(full_dictionary, dictionary_inbreast)

save_json("full_dictionary_header", full_dictionary, f"../../metadata/dictionary_dicom")

## Metadados arquivos CSV e TXT

Arquivo CSV - CMMD

In [ ]:
metadata_cmmd = load_json("csv_and_dicom_CMMD", "../../metadata/extracted_metadata")
               
dictionary_metadata = create_dict_meta(metadata_cmmd, 'csv')
save_json("dictionary_csv_CMMD", dictionary_metadata, f"../../metadata/dictionary_csv_txt")

Arquivo CSV - DDSM

In [ ]:
metadata_files = ['calc_case_description_test_set',
             'mass_case_description_train_set',
             'calc_case_description_train_set',
             'mass_case_description_test_set']

dictionary_metadata = {}

for current_meta in metadata_files:
    metadata_ddsm = load_json(current_meta, "../../metadata/extracted_metadata/csv_and_dicom_DSSM")
    
    dictionary_metadata = merge_dictionary(dictionary_metadata, create_dict_meta(metadata_ddsm, 'csv'))
    
save_json("dictionary_csv_DDSM", dictionary_metadata, f"../../metadata/dictionary_csv_txt")    

Arquivo CSV - INBREAST

In [ ]:
metadata_inbreast = load_json("csv_and_dicom_INBREAST", "../../metadata/extracted_metadata")

dictionary_metadata = create_dict_meta(metadata_inbreast, 'csv')
save_json("dictionary_csv_INBREAST", dictionary_metadata, f"../../metadata/dictionary_csv_txt")

Arquivo txt - MIAS

In [ ]:
metadata_mias = load_json("txt_and_pgm_mias", "../../metadata/extracted_metadata")
               
dictionary_metadata = create_dict_meta(metadata_mias, 'txt')
save_json("dictionary_txt_MIAS", dictionary_metadata, f"../../metadata/dictionary_csv_txt")

Juntar Dicionários de metadados CSV - DDSM e CMMD

In [ ]:
dictionary_cmmd = load_json("dictionary_csv_CMMD", "../../metadata/dictionary_csv_txt")
dictionary_ddsm = load_json("dictionary_csv_DDSM", "../../metadata/dictionary_csv_txt")
dictionary_inbreast = load_json("dictionary_csv_INBREAST", "../../metadata/dictionary_csv_txt")

full_dictionary = merge_dictionary(dictionary_cmmd, dictionary_ddsm)
full_dictionary = merge_dictionary(full_dictionary, dictionary_inbreast)

save_json("full_dictionary_csv", full_dictionary, f"../../metadata/dictionary_Csv_txt")